In [29]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import os
from glob import glob
import tqdm
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


In [30]:
train_path = '../carvana/carvana-image-masking-challenge/train/train'
train_masks_path = '../carvana/carvana-image-masking-challenge/train_masks/train_masks'

In [31]:
file_list = os.listdir(train_path)
mask_list = os.listdir(train_masks_path)


In [64]:
# create a feature matrix
num = 500
feature =[]
for i in tqdm.tqdm(range(num)):
    
    imgname = os.path.join(train_path,file_list[i])
    maskname = os.path.join(train_masks_path,file_list[i][:-4] + '_mask.gif')
    
    print(imgname)
    img = image.load_img((imgname), target_size=(256,256))
    img = image.img_to_array(img,data_format='channels_last')
    
    mask = image.load_img((maskname), target_size=(256,256))
    mask = image.img_to_array(mask,data_format='channels_last')
    feature.append((img , mask))
    

In [ ]:
# imgname = '../carvana/carvana-image-masking-challenge/baby.JPG'
# img = image.load_img((imgname), target_size=(256,256))
# img = image.img_to_array(img ,data_format='channels_last')
# img

In [33]:
# feature = np.array(feature)
# feature.shape
data = feature 

feature[300][1].shape

(256, 256, 3)

In [58]:
# training , testing split 

def split(X,r):
    np.random.shuffle(X)
    
    test_size = int(r*len(X))
    train_size = len(X) - test_size
        
    train_x =[]
    train_y =[]
    for i in range(train_size):
        train_x.append(X[i][0])
        train_y.append(X[i][1][:,:,0].reshape((256,256,1)))
        
    test_x=[] 
    test_y=[] 
    for i in range(test_size):
        test_x.append(X[i][0])
        test_y.append(X[i][1][:,:,0].reshape((256,256,1)))
        
    return np.array(train_x),np.array(train_y)/255.0,np.array(test_x),np.array(test_y)/255.0   
        
        

In [59]:
train_x,train_y,test_x,test_y = split(data,0.2)


(100, 256, 256, 1)
(100, 256, 256, 1)


In [63]:
outfile = 'trainyfile.npy'
np.save(outfile, train_y)

print(train_y.shape)
h = np.load(outfile)
print(h.shape)

(400, 256, 256, 1)
(400, 256, 256, 1)


In [36]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(x=train_x,y=train_y,batch_size=5)

test_generator = train_datagen.flow(x=test_x,y=test_y,batch_size=5)

test_generator = train_datagen.flow(x=test_x,y=test_y,batch_size=5)



In [37]:
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.optimizers import SGD, Adam,Adagrad
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from math import sqrt
from keras.callbacks import History 
from keras.optimizers import Adam, SGD
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, merge, concatenate
from keras.layers import Activation, Dropout, Flatten, Dense,Input
from keras.layers import BatchNormalization
from keras.models import Model
from keras.activations import relu
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import ELU
from keras.callbacks import History 


In [38]:
def unet(pretrained_weights = None,input_size = (256,256,3)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
#     conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
#     conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
#     conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
#     conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
#     conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9) #(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [39]:
test_y[0].shape

(256, 256, 1)

In [67]:
# del model
epochs = 3
model = unet()
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, epsilon=1e-5, patience=3, verbose=1)

model.fit_generator(train_generator,steps_per_epoch=len(train_x)/5,validation_steps= len(test_x)/5,epochs=epochs,validation_data=test_generator,callbacks = [lr_reduce],verbose=1)

In [56]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_96 (Conv2D)              (None, 256, 256, 64) 1792        input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_97 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_96[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_21 (MaxPooling2D) (None, 128, 128, 64) 0           conv2d_97[0][0]                  
__________________________________________________________________________________________________
conv2d_98 

In [ ]:
plt.imshow([])